<a href="https://colab.research.google.com/github/avlis-MMO/Deep_Learning/blob/main/Project_4_Image_Classification/CNN_Trasnf_Learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Libraries**

In [ ]:
!pip install matplotlib
!pip install tensorflow
!pip install numpy

# **Import Libraries**

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import random


# **1. Data Preparation**

In [ ]:
data = tf.keras.datasets.cifar10.load_data()
(x_train, y_train), (x_test, y_test) = data

split_classes = [8, 9]

# Check data distribution too see if tis uniform
for i in range(0,10):
    print(np.count_nonzero(y_train==i))

# Atribute labels
data_dict = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck'
}

# Visualize some of the images
n = random.randrange(5000)
plt.imshow(x_train[n])
label_text = data_dict[y_train[n][0]]
plt.text(0.5, -0.1, label_text, ha='center', transform=plt.gca().transAxes, fontsize=12)

plt.show()

# **2. Preprocess**

In [ ]:
# Reduce the values to be between 0 and 1 to make the model better

x_train = (x_train/255).astype(np.float32)
x_test = (x_test/255).astype(np.float32)

# Separate into two classes to make two models
# 8 classes
mask_train_8 = np.isin(y_train, split_classes, invert=True).flatten()
mask_test_8 = np.isin(y_test, split_classes, invert=True).flatten()

x_train_8, y_train_8 = x_train[mask_train_8], y_train[mask_train_8]
x_test_8, y_test_8 = x_test[mask_test_8], y_test[mask_test_8]


# 2 classes
mask_train_2 = np.isin(y_train, split_classes).flatten()
mask_test_2 = np.isin(y_test, split_classes).flatten()

x_train_2, y_train_2 = x_train[mask_train_2], y_train[mask_train_2]
x_test_2, y_test_2 = x_test[mask_test_2], y_test[mask_test_2]

y_train_2 = np.isin(y_train_2, split_classes[0]).astype(int)
y_test_2 = np.isin(y_test_2, split_classes[0]).astype(int)

input_shape = (32, 32, 3)


# 3. **Build Model**

In [ ]:
# Mode from where we will transfer data

x = tf.random.normal(input_shape)
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2), 1),
        tf.keras.layers.Conv2D(64, (2,2), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2), 1),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(8, activation='softmax')])

single_image = x_train[0]
single_image = np.expand_dims(single_image, axis=0)


# **4. Compile Model**

In [ ]:
learning_rate = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate)

model.compile(optimizer= optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

# **5. Train Model**

In [ ]:
model.fit(x_train_8,y_train_8, batch_size=64, epochs=10, validation_data=(x_test_8, y_test_8))


# **6. Build Second Model**

In [ ]:
model2 = tf.keras.Sequential([

        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2), 1),
        tf.keras.layers.Conv2D(64, (2,2), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2), 1),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')])

model2.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])

model2.fit(x_train_2,y_train_2, batch_size=64, epochs=10, validation_data=(x_test_2, y_test_2))


# **7. Transfer Learning**

In [ ]:
model.layers[0].trainable = False
model.layers[1].trainable = False
model.layers[2].trainable = False

model.pop()
model.add(tf.keras.layers.Dense(1 ,activation="sigmoid"))
model.layers





In [ ]:
model.compile(optimizer= "adam", loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train_2, y_train_2, batch_size=64, epochs=10, validation_data=(x_test_2, y_test_2))